In [1]:
'''main'''
import pandas as pd
import numpy as np
import os

'''Data viz'''
import matplotlib.pyplot as plt
import seaborn as sns 
colors = sns.color_palette()
import matplotlib as mpl

%matplotlib inline
'''Data Prep'''
import sklearn.preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score

'''Algos'''
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/asthma-disease-dataset/asthma_disease_data.csv


**Details**

This code is not optimised to give the best performance from the models selected rather it just shows the important steps needed to get better results from models amd been able to predict good data. it focuses on having good accurecy, precision and recall. 

In [2]:
data = pd.read_csv(os.path.join(dirname, filename))

In [3]:
asthma_data = pd.DataFrame(data)
asthma_data.head(3)

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,PhysicalActivity,DietQuality,SleepQuality,...,LungFunctionFEV1,LungFunctionFVC,Wheezing,ShortnessOfBreath,ChestTightness,Coughing,NighttimeSymptoms,ExerciseInduced,Diagnosis,DoctorInCharge
0,5034,63,0,1,0,15.848744,0,0.894448,5.488696,8.701003,...,1.369051,4.941206,0,0,1,0,0,1,0,Dr_Confid
1,5035,26,1,2,2,22.757042,0,5.897329,6.341014,5.153966,...,2.197767,1.702393,1,0,0,1,1,1,0,Dr_Confid
2,5036,57,0,2,1,18.395396,0,6.739367,9.196237,6.840647,...,1.698011,5.022553,1,1,1,0,1,1,0,Dr_Confid


In [4]:
#Checking the statistics like youngest age and oldest age 
asthma_data.describe()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,PhysicalActivity,DietQuality,SleepQuality,...,GastroesophagealReflux,LungFunctionFEV1,LungFunctionFVC,Wheezing,ShortnessOfBreath,ChestTightness,Coughing,NighttimeSymptoms,ExerciseInduced,Diagnosis
count,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,...,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000
mean,6229.500000,42.137960,0.493311,0.669732,1.307274,27.244877,0.141722,5.051786,5.022867,7.019012,...,0.158027,2.548564,3.741270,0.596154,0.500418,0.503344,0.503344,0.602425,0.604933,0.051839
std,690.655244,21.606655,0.500060,0.986120,0.898242,7.201628,0.348838,2.903574,2.909980,1.732475,...,0.364842,0.861809,1.303689,0.490770,0.500104,0.500093,0.500093,0.489499,0.488967,0.221749
min,5034.000000,5.000000,0.000000,0.000000,0.000000,15.031803,0.000000,0.001740,0.003031,4.001437,...,0.000000,1.000459,1.500045,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5631.750000,23.000000,0.000000,0.000000,1.000000,20.968313,0.000000,2.578333,2.432043,5.498500,...,0.000000,1.824113,2.607489,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6229.500000,42.000000,0.000000,0.000000,1.000000,27.052202,0.000000,5.016881,5.115383,6.975839,...,0.000000,2.553244,3.734982,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
75%,6827.250000,61.000000,1.000000,1.000000,2.000000,33.555903,0.000000,7.540234,7.544216,8.526950,...,0.000000,3.292897,4.864121,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7425.000000,79.000000,1.000000,3.000000,3.000000,39.985611,1.000000,9.995809,9.999904,9.996235,...,1.000000,3.999719,5.999421,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
#removing unwanted columns
original_data = asthma_data.copy()
asthma_data = asthma_data.drop(['PatientID', 'DoctorInCharge'], axis=1)

In [6]:
#checking for missing values 
asthma_data.isna().sum()

Age                       0
Gender                    0
Ethnicity                 0
EducationLevel            0
BMI                       0
Smoking                   0
PhysicalActivity          0
DietQuality               0
SleepQuality              0
PollutionExposure         0
PollenExposure            0
DustExposure              0
PetAllergy                0
FamilyHistoryAsthma       0
HistoryOfAllergies        0
Eczema                    0
HayFever                  0
GastroesophagealReflux    0
LungFunctionFEV1          0
LungFunctionFVC           0
Wheezing                  0
ShortnessOfBreath         0
ChestTightness            0
Coughing                  0
NighttimeSymptoms         0
ExerciseInduced           0
Diagnosis                 0
dtype: int64

In [7]:
#scaling the dataframe and spliting it into target data and scaled feature data
standard_scaler = pp.StandardScaler()
features_To_Scale = asthma_data.drop('Diagnosis', axis=1)
scald_features = standard_scaler.fit_transform(features_To_Scale)


scaled_asthma_data = pd.DataFrame(scald_features, columns=features_To_Scale.columns)
target_data = asthma_data['Diagnosis']

In [8]:
#over sampling the minority data points 
from imblearn.over_sampling import RandomOverSampler
random_over_sample = RandomOverSampler(random_state=0)
overSampled_data_X, overSampled_data_y = random_over_sample.fit_resample(scaled_asthma_data, target_data)

In [9]:
#spliting data between training and testing data
X_train, X_test, y_train, y_test = train_test_split(overSampled_data_X, overSampled_data_y, test_size=0.33, random_state=42, stratify=overSampled_data_y)

In [10]:
#creating initializing the models with default values
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'Support vector machines': SVC()
}

In [11]:
stratifiedKFold_cross_validation = StratifiedKFold(n_splits=5, random_state=24, shuffle=True)

In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score as acc_score 



accuracies = [] 
precisions = []
recalls = []

skf = StratifiedKFold(n_splits=5)

for train_index, validation_index in skf.split(X_train, y_train):
    for name, model in models.items():

        X_trn = X_train.iloc[train_index]
        X_val = X_train.iloc[validation_index]
        y_trn = y_train.iloc[train_index]
        y_val = y_train.iloc[validation_index]

        model.fit(X_trn, y_trn)

        y_pred = model.predict(X_val)

        accuracy = acc_score(y_val, y_pred)
        precision, recall, thresholds = precision_recall_curve(y_val, y_pred)

        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)

        print(f'Fold accuracy for {name}: {accuracy}')
        print(f'Fold precision for {name}: {precision}')
        print(f'Fold recall for {name}: {recall}')
     
        

print(f'\nAverage Accuracy: {sum(accuracies)/len(accuracies)}')


Fold accuracy for Logistic Regression: 0.5641447368421053
Fold precision for Logistic Regression: [0.5       0.5631068 1.       ]
Fold recall for Logistic Regression: [1.         0.57236842 0.        ]
Fold accuracy for Random Forest: 1.0
Fold precision for Random Forest: [0.5 1.  1. ]
Fold recall for Random Forest: [1. 1. 0.]
Fold accuracy for Support vector machines: 0.9457236842105263
Fold precision for Support vector machines: [0.5        0.92211838 1.        ]
Fold recall for Support vector machines: [1.         0.97368421 0.        ]
Fold accuracy for Logistic Regression: 0.5608552631578947
Fold precision for Logistic Regression: [0.5       0.5576324 1.       ]
Fold recall for Logistic Regression: [1.         0.58881579 0.        ]
Fold accuracy for Random Forest: 1.0
Fold precision for Random Forest: [0.5 1.  1. ]
Fold recall for Random Forest: [1. 1. 0.]
Fold accuracy for Support vector machines: 0.96875
Fold precision for Support vector machines: [0.5        0.94117647 1.     